# Ejercicio 1

## Creación de una bases de datos de películas extraídas de un API

En este ejercicio vamos a poner en práctica lo que hemos aprendido sobre la extracción de contenido de APIs y la creación de una base de datos con sus tablas. Siguiendo las fases propuestas, conseguiremos extraer datos de películas, almacenarlos en un DataFrame y finalmente crear una base de datos con la estructura adecuada.

Fase 1: Extracción de datos de películas

Nuestro primer objetivo es extraer los datos de películas de la API. En Adalab nos hemos creado un API
muy sencilla que contiene la siguiente información:
Datos a extraer:
- Título
- Año de lanzamiento
- Duración (en minutos)
- Género
- Contenido para adultos (sí o no)


El objetivo es extraer 100 películas de esta API utilizando el siguiente endpoint:
https://beta.adalab.es/resources/apis/pelis/pelis.json

In [ ]:
import requests
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error

In [ ]:
url_films = "https://beta.adalab.es/resources/apis/pelis/pelis.json"
films = requests.get(url_films)

In [ ]:
films.status_code

In [ ]:
film_data = films.json()
film_data

In [ ]:
df_film_data = pd.DataFrame(film_data)
df_film_data

## Creación de la Base de Datos
En esta fase se creará la base de datos utilizando Python

In [ ]:
try:
    cnx = mysql.connector.connect(
        host='localhost',
        user='root',
        password='AlumnaAdalab',
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS films_db"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

In [ ]:
mycursor.execute("USE films_db")

query = '''CREATE TABLE IF NOT EXISTS films(
	idFilm INT,
    title VARCHAR(70) NOT NULL,
    year YEAR,
    runtime INT,
    genre VARCHAR(45),
    rating BOOLEAN,
    PRIMARY KEY (idFilm)
);'''
mycursor.execute(query)

query = '''CREATE TABLE IF NOT EXISTS languages(
    idLang INT AUTO_INCREMENT,
    language VARCHAR(45) UNIQUE NOT NULL,
    PRIMARY KEY (idLang)
);'''
mycursor.execute(query)

query = '''CREATE TABLE IF NOT EXISTS subtitles_films(
    idFilm INT NOT NULL,
    idLang INT NOT NULL,
    PRIMARY KEY (idFilm, idLang),
    FOREIGN KEY (idFilm) REFERENCES films(idFilm),
    FOREIGN KEY (idLang) REFERENCES languages(idLang)
);'''

mycursor.execute(query)

# Inserción de los Datos en la Base de Datos

En este apartado vamos a realizar la inserción de los datos extraídos en las tablas creadas en nuestra base de datos MySQL.

In [ ]:
mycursor.execute("USE films_db")

for index, film in df_film_data.iterrows(): 
    
    query_insert = '''INSERT INTO films (idFilm, title, year, runtime, genre, rating) 
                      VALUES (%s, %s, %s, %s, %s, %s);
                      '''
    
    values = (
        film['id'], 
        film['titulo'], 
        film['año'], 
        film['duracion'], 
        film['genero'], 
        film['adultos']
    )
    
    mycursor.execute(query_insert, values)

    idFilm = film['id']

    for subtitle in film['subtitulos']:       
        
        idLang = None
        query_insert_lang = '''INSERT IGNORE INTO languages (language) 
                            VALUES (%s);
                            '''
        # con el IGNORE aunque se ignore el valor que se repite, el autoincrement reserva un ID para ese valor ignorado.
        
        mycursor.execute(query_insert_lang, (subtitle,))

        query_select_lang = """SELECT idLang
                            FROM languages
                            WHERE language = %s
                            """
        mycursor.execute(query_select_lang, (subtitle,))        
        
        idLang = mycursor.fetchone()[0]

        query_insert_relation = '''INSERT INTO subtitles_films (idFilm, idLang) 
                                VALUES (%s, %s)'''
        
        valuesLang = (
            idFilm, 
            idLang
        )
        
        mycursor.execute(query_insert_relation, valuesLang)


cnx.commit()
cnx.close()
